In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap

df_airbnb = pd.read_csv('listings.csv')

In [ ]:
df_airbnb.shape

In [ ]:
df_airbnb.columns

In [ ]:
df_airbnb.head()

In [ ]:
df_airbnb.dtypes

In [ ]:
df_airbnb.isna().sum()

In [ ]:
df_airbnb = df_airbnb.drop(['name', 'host_name', 'neighbourhood_group', 'last_review', 'reviews_per_month', 'license'], axis=1)

In [ ]:
df_airbnb['price'] = df_airbnb['price'].fillna(0)

In [ ]:
df_airbnb.isna().sum()

In [ ]:
# Criar mapa base centrado no RJ
mapa_rj = folium.Map(location=[-22.9068, -43.1729], zoom_start=11)

# Criar cluster de marcadores
marker_cluster = MarkerCluster().add_to(mapa_rj)

# Adicionar marcadores ao cluster para todos os registros
for _, row in df_airbnb.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=(
            f"Preço: R${row['price']:.2f}<br>"
            f"Bairro: {row['neighbourhood']}<br>"
            f"Tipo: {row['room_type']}"
        ),
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(marker_cluster)

# Exibir mapa
mapa_rj

In [ ]:
#Função para definir cores para faixas de valores
def definir_cor(preco):
    if preco <= 500:
        return 'green'
    elif preco <= 1000:
        return 'orange'
    else:
        return 'red'

mapa_preco = folium.Map(location=[-22.9068, -43.1729], zoom_start=11)

# Adicionar apenas marcadores com cor, sem popups e com preço > 0
for _, row in df_airbnb.iterrows():
    preco = row['price']
    if preco > 0:
        cor = definir_cor(preco)
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3.5,
            color=cor,
            fill=True,
            fill_color=cor,
            fill_opacity=0.7,
            weight=0.5
        ).add_to(mapa_preco)

# Exibir o mapa
mapa_preco

In [ ]:
# Filtrar imóveis com avaliações
df_reviews = df_airbnb[df_airbnb['number_of_reviews'] > 0]

# Criar mapa base
mapa_reviews = folium.Map(location=[-22.9068, -43.1729], zoom_start=11)

# Adicionar camada HeatMap ponderada por número de avaliações
HeatMap(
    data=df_reviews[['latitude', 'longitude', 'number_of_reviews']].values,
    radius=10,
    max_zoom=13,
    blur=15
).add_to(mapa_reviews)

# Exibir o mapa
mapa_reviews